In [42]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [43]:
import copy
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [44]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
# command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)
sar = celeri.process_sar(sar, command)
segment, station, block, mogi, sar = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = celeri.create_assembly_dictionary()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
partials = {} 
assembly, partials["block_motion_constraints"] = celeri.block_constraints(assembly, block, command)
assembly, partials["slip_rate_constraints"] = celeri.slip_rate_constraints(assembly, segment, block, command)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# celeri.plot_block_labels(segment, block, station)
plt.figure()
for i in range(len(segment)):
    plt.plot(
        [segment.lon1.values[i], segment.lon2.values[i]],
        [segment.lat1.values[i], segment.lat2.values[i]],
        "-k",
        linewidth=0.5,
    )
plt.plot(segment.lon1, segment.lat1, "r.", markersize=10)
plt.plot(segment.lon2, segment.lat2, "r.", markersize=10)
plt.show()

In [ ]:
segment_split = segment.copy(deep=True)
segment_split = celeri.split_segments_crossing_meridian(segment_split)
segment_split = segment_split.reset_index(drop=True)

plt.figure()
for i in range(len(segment_split)):
    plt.plot(
        [segment_split.lon1.values[i], segment_split.lon2.values[i]],
        [segment_split.lat1.values[i], segment_split.lat2.values[i]],
        "-k",
        linewidth=0.5,
    )
plt.plot(segment_split.lon1, segment_split.lat1, "r.", markersize=10)
plt.plot(segment_split.lon2, segment_split.lat2, "r.", markersize=10)

plt.show()

In [ ]:
print(len(segment))
print(len(segment_split))
